In [1]:
from graph import *
from models import *
from metrics import *

import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import plotly.graph_objects as go

In [2]:
from copy import deepcopy

In [3]:
# load data from the spreadsheet which defines the structure of the workflow,
# as well as the parameters for data rates, efficiency, data reduction, and classifier performance
run3_system = dataframes_from_spreadsheet("cms_system_60.xlsx")
run5_system = dataframes_from_spreadsheet("cms_system_200.xlsx")

In [4]:
run5_system.detectors

,Category,Detector,Data (bytes),Sample Rate,Compression,Link Efficiency (J/bit),Op Efficiency (J/op),PU 200
0,Tracking,Inner Tracker,1440000,40000000,0,2.220000e-11,0,1.440
1,Tracking,Outer Tracker PS,720000,40000000,0,2.220000e-11,0,0.720
2,Tracking,Outer Tracker 2S,430000,40000000,0,2.220000e-11,0,0.430
3,Tracking,Track Finder TPG,10000,40000000,0,2.220000e-11,0,0.010
4,Timing,MIP Timing BTL,240000,40000000,0,2.220000e-11,0,0.240
5,Timing,MIP Timing ETL,440000,40000000,0,2.220000e-11,0,0.440
6,Calorimetry,ECAL Barrel,600000,40000000,0,2.220000e-11,0,0.600
7,Calorimetry,HCAL Barrel,240000,40000000,0,2.220000e-11,0,0.240
8,Calorimetry,HCAL HO,30000,40000000,0,2.220000e-11,0,0.030
9,Calorimetry,HCAL HF,60000,40000000,0,2.220000e-11,0,0.060


In [5]:
#import the data predicting wall time scaling by pileup
scaling = pd.read_excel("wall time scaling.xlsx", sheet_name="Data")
#fit a polynomial to this data for CPU and GPU runtimes
fit_poly = lambda x, k3, k2, k1: k3 * x ** 3 + k2 * x ** 2 + k1 * x
k, cv = curve_fit(fit_poly, scaling["Size"], scaling["Wall Time"])
k_gpu, cv_gpu = curve_fit(fit_poly, scaling["Size"], scaling["Wall Time GPU"])

In [6]:
#define a dictionary with functions defining the scaling of trigger runtimes with incoming data
funcs = {"Global": lambda x: fit_poly(x, *k), "Intermediate": lambda x: x / 2.0e6}
funcs_gpu = {"Global": lambda x: fit_poly(x, *k_gpu), "Intermediate": lambda x: x / 2.0e6}

In [7]:
baseline_r3 = construct_graph(run3_system.detectors, run3_system.triggers, run3_system.globals, funcs)

/Users/wilkie/code/system_flow/classifier.py:200: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  fit = lambda l: np.abs(self.egamma_rate - quad(lambda x: self.exp_dist(x, l) * interpolator(x), np.min(xs), np.max(xs))[0])


In [8]:
baseline = construct_graph(run5_system.detectors, run5_system.triggers, run5_system.globals, funcs)

/Users/wilkie/code/system_flow/classifier.py:200: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  fit = lambda l: np.abs(self.egamma_rate - quad(lambda x: self.exp_dist(x, l) * interpolator(x), np.min(xs), np.max(xs))[0])
/Users/wilkie/code/system_flow/classifier.py:424: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probabl

In [9]:
baseline.nodes["Intermediate"]

{'type': 'processor',
 'reduction ratio': 53.3,
 'classifier': <classifier.L1TClassifier at 0x31348cf70>,
 'data reduction': 1.0,
 'op efficiency': 0.003,
 'sample data': 260000,
 'complexity': <function __main__.<lambda>(x)>,
 'global ratio': 5330.0,
 'message size': 8425000.0,
 'ops': 4.2125,
 'input rate': 39999999,
 'error matrix': array([[0.9813, 0.4969],
        [0.0187, 0.5031]]),
 'contingency': array([[39244635,     3728],
        [  747859,     3775]]),
 'discards': array([39244635,     3728]),
 'output rate': 751634,
 'energy': 0.012637500000000001,
 'power': 505499.98736250005}

In [10]:
baseline.nodes["Intermediate"]["energy"] * np.sum(baseline.nodes["Intermediate"]["discards"])

496001.18741250003

In [11]:
a1 = list(baseline.predecessors("Intermediate"))

In [12]:
a1

['Tracking', 'Timing', 'Calorimetry', 'Muon']

In [13]:
list(baseline.successors("Intermediate"))

['Global']

In [14]:
"Intermediate" in baseline.nodes.keys()

True

In [15]:
"""
Return the amount of energy expended by the system to reach the current node
"""
def upstream_energy(graph, node):
    def get_energy(node):
        if "energy" in node.keys():
            return node["energy"]
        else:
            return 0.0
    
    def traverse(start):
        up = list(graph.predecessors(start))

        if len(up) == 0:
            return get_energy(graph.nodes[start])
        else:
            return get_energy(graph.nodes[start]) + functools.reduce(lambda x, y: x + y, map(traverse, up))
    
    return traverse(node)

In [16]:
upstream_energy(baseline, "Intermediate")

0.012637500000000001

In [17]:
baseline.graph["Root Node"]

'Disk'

In [18]:
baseline.nodes

NodeView(('Inner Tracker', 'Outer Tracker PS', 'Outer Tracker 2S', 'Track Finder TPG', 'MIP Timing BTL', 'MIP Timing ETL', 'ECAL Barrel', 'HCAL Barrel', 'HCAL HO', 'HCAL HF', 'HGCAL', 'HGCAL TPG Stage1', 'HGCAL TPG Stage2', 'Muon DT', 'Muon CSC', 'Muon GEM GE1', 'Muon GEM GE2', 'Muon GEM ME0', 'Muon RPC', 'Tracking', 'Timing', 'Calorimetry', 'Muon', 'Intermediate', 'Global', 'Disk'))

In [19]:
upstream_energy(baseline, "Disk")

446.05484938484136

In [20]:
def quantify_error_cost(graph):
    #the cost of a true positive is the cost to get to the final node
    positive = upstream_energy(graph, graph.graph["Root Node"])
    #the cost of a true negative is the average energy for a discarded message
    classifiers = active_classifiers(graph)
    energy = [upstream_energy(graph, c) for c in classifiers]
    negatives = [np.sum(graph.nodes[c]["discards"]) for c in classifiers]
    negative = np.average(energy, weights=negatives)

    return (negative, positive)

In [21]:
#quantify_error_cost(baseline)

In [22]:
baseline.nodes["Inner Tracker"]["global ratio"]

5330.0

In [23]:
(5330 * 8.3) / 446

99.19058295964128

In [24]:
5330 * 8.3

44239.00000000001

In [25]:
baseline.nodes["Global"]["discards"]

array([743371,    736])

In [26]:
baseline.nodes["Intermediate"]["energy"] * baseline.nodes["Intermediate"]["output rate"]

9498.774675

In [27]:
gpu = construct_graph(run5_system.detectors, run5_system.triggers, run5_system.globals, funcs_gpu)

/Users/wilkie/code/system_flow/classifier.py:200: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  fit = lambda l: np.abs(self.egamma_rate - quad(lambda x: self.exp_dist(x, l) * interpolator(x), np.min(xs), np.max(xs))[0])
/Users/wilkie/code/system_flow/classifier.py:424: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probabl

In [28]:
gpu.nodes["Disk"]

{'type': 'storage',
 'reduction ratio': 1.0,
 'classifier': <classifier.DummyClassifier at 0x313448bb0>,
 'data reduction': 1.0,
 'op efficiency': 0.0,
 'sample data': 0,
 'complexity': <function graph.triggers.<locals>.<lambda>(x)>,
 'global ratio': 1.0,
 'message size': 8425000.0,
 'ops': 8425000.0,
 'input rate': 7603,
 'error matrix': array([[0., 0.],
        [1., 1.]]),
 'contingency': array([[   0,    0],
        [4711, 2892]]),
 'discards': array([0, 0]),
 'output rate': 7603,
 'energy': 0.0,
 'power': 0.0}

In [29]:
baseline_2 = deepcopy(baseline)
baseline_2.nodes["Intermediate"]["reduction ratio"] = 400
baseline_2 = update_throughput(baseline_2)


In [30]:
l1t = deepcopy(baseline)
l1t.nodes["Intermediate"]["classifier"].skill_boost = 0.40
l1t = update_throughput(l1t)

In [31]:
smpx = deepcopy(baseline)
smpx.nodes["Inner Tracker"]["sample data"] *= (1 - 0.54)
smpx = update_throughput(smpx)

In [32]:
gpu_smpx = deepcopy(gpu)
gpu_smpx.nodes["Inner Tracker"]["sample data"] *= (1 - 0.54)
gpu_smpx = update_throughput(gpu_smpx)

In [33]:
gpu_l1t = deepcopy(gpu)
gpu_l1t.nodes["Intermediate"]["classifier"].skill_boost = 0.40
gpu_l1t = update_throughput(gpu_l1t)

In [34]:
smpx_l1t = deepcopy(baseline)
smpx_l1t.nodes["Inner Tracker"]["sample data"] *= (1 - 0.54)
smpx_l1t.nodes["Intermediate"]["classifier"].skill_boost = 0.40
smpx_l1t = update_throughput(smpx_l1t)

In [35]:
gpu_smpx_l1t = deepcopy(gpu)
gpu_smpx_l1t.nodes["Intermediate"]["classifier"].skill_boost = 0.40
gpu_smpx_l1t.nodes["Inner Tracker"]["sample data"] *= (1 - 0.54)
gpu_smpx_l1t = update_throughput(gpu_smpx_l1t)

In [36]:
np.sum(gpu.graph["performance"][:,1])

7502

In [37]:
gpu.nodes["Intermediate"]

{'type': 'processor',
 'reduction ratio': 53.3,
 'classifier': <classifier.L1TClassifier at 0x31352bc40>,
 'data reduction': 1.0,
 'op efficiency': 0.003,
 'sample data': 260000,
 'complexity': <function __main__.<lambda>(x)>,
 'global ratio': 5330.0,
 'message size': 8425000.0,
 'ops': 4.2125,
 'input rate': 39999999,
 'error matrix': array([[0.9813, 0.5241],
        [0.0187, 0.4759]]),
 'contingency': array([[39244635,     3932],
        [  747859,     3571]]),
 'discards': array([39244635,     3932]),
 'output rate': 751430,
 'energy': 0.012637500000000001,
 'power': 505499.98736250005}

In [38]:
has_classifier(gpu.nodes["Intermediate"])

True

In [39]:
ac = active_classifiers(gpu)

In [40]:
[downstream_classifier(gpu, c) for c in ac]

[True, False]

In [41]:
gpu.nodes["Intermediate"]["contingency"]

array([[39244635,     3932],
       [  747859,     3571]])

In [42]:
gpu.nodes["Global"]["contingency"]

array([[743147,    678],
       [  4711,   2892]])

In [43]:
pipeline_contingency(gpu)

array([[39987782,     4610],
       [    4711,     2892]])

In [44]:
precision(gpu.nodes["Intermediate"]["contingency"])

0.004752272334083015

In [45]:
recall(gpu.nodes["Intermediate"]["contingency"])

0.475942956150873

In [46]:
gpu.graph

{'globals':    Year
 0  2032,
 'Root Node': 'Disk',
 'link power': 124603.128972,
 'op power': 168090248.384996,
 'performance': array([[39987782,     4610],
        [    4711,     2892]])}

In [47]:
def extract_results(graph):
    power = (graph.graph["op power"] + graph.graph["link power"]) / density_scale_model(2032)
    confusion = graph.graph["performance"]
    acc = precision(confusion)
    rec = recall(confusion)
    f1 = f1_score(confusion)
    prod = f1 * np.sum(get_passed(confusion)) / power


    return power, acc, rec, f1, prod

In [48]:
conditions = [baseline_r3, baseline_2, baseline]

In [49]:
pileup = np.array([60, 200, 200])[:,np.newaxis]
rejection = np.array([400, 400, 53])[:,np.newaxis]

In [50]:
pileup.shape

(3, 1)

In [51]:
cond_results = np.stack([extract_results(g) for g in conditions])

In [52]:
cond_results = np.concatenate((pileup, rejection, cond_results), axis=1)

In [53]:
df2 = pd.DataFrame(cond_results, columns = ["Pileup", "L1T Reduction Ratio", "Power (W)", "Accuracy (%)", "Recall (%)", "F1 Score (%)", "Productivity (Relevant Samples/J)"])

In [54]:
df2

,Pileup,L1T Reduction Ratio,Power (W),Accuracy (%),Recall (%),F1 Score (%),Productivity (Relevant Samples/J)
0,60.0,400.0,3.223989e+05,0.275449,0.276553,0.276000,0.000858
1,200.0,400.0,6.979455e+06,0.233268,0.237475,0.235353,0.000034
2,200.0,53.0,5.167889e+07,0.403721,0.404959,0.404339,0.000059


In [55]:
extract_results(baseline_2)[-1] * 1000

0.034260292586751055

In [56]:
extract_results(baseline)[-1] * 1000

0.05887606809074921

In [57]:
extract_results(baseline_r3)[-1] * 1000

0.8577945278104697

In [58]:
extract_results(baseline_2)[-1] * 1000

0.034260292586751055

In [59]:
extract_results(baseline)[-1] * 1000

0.05887606809074921

In [60]:
all_graphs = [baseline_r3, baseline, gpu, l1t, smpx, gpu_l1t, smpx_l1t, gpu_smpx, gpu_smpx_l1t]

In [61]:
pileup = np.array([[60, 200, 200, 200, 200, 200, 200, 200, 200,],])
rejection = np.array([[400, 53, 53, 53, 53, 53, 53, 53, 53],])
has_gpu = [False, False, True, False, False, True, False, True, True]
has_smpx = [False, False, False, False, True, False, True, True, True]
has_l1t = [False, False, False, True, False, True, True, False, True]

In [62]:
results = np.stack([extract_results(g) for g in all_graphs])

In [63]:
results

array([[3.22398886e+05, 2.75449102e-01, 2.76553106e-01, 2.76000000e-01,
        8.57794528e-04],
       [5.16788909e+07, 4.03720930e-01, 4.04958678e-01, 4.04338857e-01,
        5.88760681e-05],
       [2.58809180e+07, 3.80376167e-01, 3.85497201e-01, 3.82919563e-01,
        1.12489728e-04],
       [5.16528809e+07, 1.00000000e+00, 7.93121834e-01, 8.84626821e-01,
        1.01901956e-04],
       [4.08977083e+07, 4.03720930e-01, 4.04958678e-01, 4.04338857e-01,
        7.43965866e-05],
       [2.60114117e+07, 1.00000000e+00, 7.95921088e-01, 8.86365323e-01,
        2.03467901e-04],
       [4.08771301e+07, 1.00000000e+00, 7.93121834e-01, 8.84626821e-01,
        1.28764656e-04],
       [2.04872869e+07, 3.80376167e-01, 3.85497201e-01, 3.82919563e-01,
        1.42104587e-04],
       [2.05905288e+07, 1.00000000e+00, 7.95921088e-01, 8.86365323e-01,
        2.57035037e-04]])

In [64]:
pileup.shape

(1, 9)

In [65]:
rejection.shape

(1, 9)

In [66]:
results = np.concatenate((pileup, rejection, np.transpose(results)), axis=0)

In [67]:
results

array([[6.00000000e+01, 2.00000000e+02, 2.00000000e+02, 2.00000000e+02,
        2.00000000e+02, 2.00000000e+02, 2.00000000e+02, 2.00000000e+02,
        2.00000000e+02],
       [4.00000000e+02, 5.30000000e+01, 5.30000000e+01, 5.30000000e+01,
        5.30000000e+01, 5.30000000e+01, 5.30000000e+01, 5.30000000e+01,
        5.30000000e+01],
       [3.22398886e+05, 5.16788909e+07, 2.58809180e+07, 5.16528809e+07,
        4.08977083e+07, 2.60114117e+07, 4.08771301e+07, 2.04872869e+07,
        2.05905288e+07],
       [2.75449102e-01, 4.03720930e-01, 3.80376167e-01, 1.00000000e+00,
        4.03720930e-01, 1.00000000e+00, 1.00000000e+00, 3.80376167e-01,
        1.00000000e+00],
       [2.76553106e-01, 4.04958678e-01, 3.85497201e-01, 7.93121834e-01,
        4.04958678e-01, 7.95921088e-01, 7.93121834e-01, 3.85497201e-01,
        7.95921088e-01],
       [2.76000000e-01, 4.04338857e-01, 3.82919563e-01, 8.84626821e-01,
        4.04338857e-01, 8.86365323e-01, 8.84626821e-01, 3.82919563e-01,
        8.8

In [68]:
df = pd.DataFrame(results.transpose(), columns = ["Pileup", "L1T Reduction Ratio", "Power (W)", "Accuracy (%)", "Recall (%)", "F1 Score (%)", "Productivity (Relevant Samples/J)"])

In [69]:
df["GPU HLT"] = has_gpu
df["L1T Tracking"] = has_l1t
df["Smart Sensors"] = has_smpx

In [70]:
df

,Pileup,L1T Reduction Ratio,Power (W),Accuracy (%),Recall (%),F1 Score (%),Productivity (Relevant Samples/J),GPU HLT,L1T Tracking,Smart Sensors
0,60.0,400.0,3.223989e+05,0.275449,0.276553,0.276000,0.000858,False,False,False
1,200.0,53.0,5.167889e+07,0.403721,0.404959,0.404339,0.000059,False,False,False
2,200.0,53.0,2.588092e+07,0.380376,0.385497,0.382920,0.000112,True,False,False
3,200.0,53.0,5.165288e+07,1.000000,0.793122,0.884627,0.000102,False,True,False
4,200.0,53.0,4.089771e+07,0.403721,0.404959,0.404339,0.000074,False,False,True
5,200.0,53.0,2.601141e+07,1.000000,0.795921,0.886365,0.000203,True,True,False
6,200.0,53.0,4.087713e+07,1.000000,0.793122,0.884627,0.000129,False,True,True
7,200.0,53.0,2.048729e+07,0.380376,0.385497,0.382920,0.000142,True,False,True
8,200.0,53.0,2.059053e+07,1.000000,0.795921,0.886365,0.000257,True,True,True


In [71]:
df.iloc[1:]

,Pileup,L1T Reduction Ratio,Power (W),Accuracy (%),Recall (%),F1 Score (%),Productivity (Relevant Samples/J),GPU HLT,L1T Tracking,Smart Sensors
1,200.0,53.0,5.167889e+07,0.403721,0.404959,0.404339,0.000059,False,False,False
2,200.0,53.0,2.588092e+07,0.380376,0.385497,0.382920,0.000112,True,False,False
3,200.0,53.0,5.165288e+07,1.000000,0.793122,0.884627,0.000102,False,True,False
4,200.0,53.0,4.089771e+07,0.403721,0.404959,0.404339,0.000074,False,False,True
5,200.0,53.0,2.601141e+07,1.000000,0.795921,0.886365,0.000203,True,True,False
6,200.0,53.0,4.087713e+07,1.000000,0.793122,0.884627,0.000129,False,True,True
7,200.0,53.0,2.048729e+07,0.380376,0.385497,0.382920,0.000142,True,False,True
8,200.0,53.0,2.059053e+07,1.000000,0.795921,0.886365,0.000257,True,True,True


In [76]:
df.iloc[1:]["Productivity (Relevant Samples/J)"] * 1000

1    0.058876
2    0.112490
3    0.101902
4    0.074397
5    0.203468
6    0.128765
7    0.142105
8    0.257035
Name: Productivity (Relevant Samples/J), dtype: float64

In [77]:
df["Productivity (Relevant Samples/J)"] * 1e3

0    0.857795
1    0.058876
2    0.112490
3    0.101902
4    0.074397
5    0.203468
6    0.128765
7    0.142105
8    0.257035
Name: Productivity (Relevant Samples/J), dtype: float64

In [74]:
df.to_excel("experimental_table.xlsx", index=False)

In [75]:
density_scale_model(2032)

6.499570514329353